## 0. 计算图与常见计算节点

神经网络可以看做是一个复杂的函数，对给定输出进行转换后给出输出；神经网络的学习需要计算损失函数关于参数的导数，现阶段是通过将神经网络的计算过程用**计算图**加以表述，通过计算图计算出损失函数关于所有参数的导数；

计算图由边和节点组成，节点对应某种特定的计算，如加法和乘法；边则对应输入节点的数据或者输出节点的数据

基于计算图通过反向传播来导数，一方面需要 **chain rule** 来作为通过局部计算来完成整体计算的依据；另一方面也需要对计算图中各个计算节点进行分类，抽象出各个节点在前向计算和反向计算时的行为特征，便于快速完成反向传播的过程

从复合函数的视角出发，我们可以将复合的各个子函数抽象为计算图的节点，每一个节点按其种类完成不同的计算任务，原则如下：

0. 每一节点只负责一种计算任务
1. 每一节点属于特定种类，在参与计算尤其是反向传播时有其特定的计算流程
2. 每一个节点输入数据的**导数形状**一定和其**自身形状**一致

这样，在搭建特定结构的神经网络时，只需要将这些节点和数据连接起来，就可以完成复杂函数的构建；且数据流向清晰明了；在反向传播时，因为每节点只负责局部的计算，只需要关注本节点的导数和上游传来的导数，就可以通过 **chain rule** 完成本节点导数的计算。

在神经网络中常见的节点包括：

1. Add 加法节点
2. Mul 乘法节点
3. Repeat 分支节点
4. Sum 累加节点
5. MatMul 矩阵乘法节点


熟练掌握这些节点的反向传播计算方法，是理解和推导神经网络反向传播的基础

另外，计算图中的计算节点均可以处理多维数据

## 1. Add 加法节点

加法节点实现的计算
$$
\mathbf{z} = \mathbf{x} + \mathbf{y}
$$

各个参数的偏导均为 1，所以加法节点就是**单纯传递上游来的导数信号**

In [1]:
def add():
    def __init__(self):
        pass  # 后向传播时不需要额外数据
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        """
        Paramters
        ---------
        dout : 上游导数信号
        """
        return dout

## 2 Mul 乘法节点

乘法节点实现的计算
$$
\mathbf{z}=\mathbf{x} \times \mathbf{y}
$$

每个参数的偏导等于另一个参数的值，所以乘法节点就是将上游导数信号乘以另一边的参数值得到当前边参数的导数

In [3]:
def Mul():
    def __init__(self):
        ## 反向计算需要保存 x 和 y 的信息
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

关于加法节点和乘法节点处理张量数据时，需要加以说明：

因为加法/乘法节点是**element-wise**的计算，所以节点的输出数据/导数信息的**形状**一定和输入数据相同，而数据数据**彼此的形状**也是相同的，所以加/乘法层在处理张量数据时，直接按标量数据的计算方式进行计算即可，**element-wise**的特性会保证各个位置上元素的正确性

## 3. Repeat 节点

Repeat 节点可以看做是一个多分支节点，一个 D 维输入数据通过 Repeate 节点被复制了 N 份，即变成了一个 N X D 的形状

从这个角度，Repeate 节点的输入数据其实是参与了 N 次计算，所以在反向求导时，需要将这 N 个导数信息进行加和，这样也保证了输入数据的导数形状和输入数据形状的一致性

典型的例子就是 Affine 层在处理批量数据时，偏置向量 $\mathbf{b}$ 实际执行了 Repeate 操作，以匹配 N 条数据，所以反向传播求关于  $\mathbf{b}$ 的导数时，就要将 N 个导数信号进行加和，以正确反映 $\mathbf{b}$ 的贡献

Repeate 也是广播功能在神经网络中的一次展示

下面只考虑向量的 Repeate 实现

In [4]:
class Repeate():
    def __init__(self):
        pass
    
    def forward(self, x):
        return np.repeat(x, N, axis = 0)
        
        
    def backward(self, dout):
        return np.sum(dout, axis=0, keepdims=True) # 保证形状是 (1, N)

## 4. Sum 节点

Sum 节点可以看做是 Repeat 节点的逆向操作，即将若干输入数据合并为一个数据输出

在反向传播时，因为 Sum 节点本质上执行的是加法操作，且 **element-wise** 的特性保证了所有输入数据和输出数据的形状一致性，所以每个输入数据的导数就是上游传递来的导数信号

下面只考虑向量的 Sum 实现

In [5]:
class Sum():
    def __init__():
        self.x_shape = None
    
    def forward(self, x):
        self.x_shape = x.shape
        return np.sum(x, axis=0, keepdims=True)
    
    def backward(self,dout):
        """
        这里需要注意，对于 dout 要执行 repeat 操作，因为 dout 的维度在 x 的基础上进行了压缩
        然后对 repeat 后的 dout 按加法节点操作即可
        """
        return np.repeat(dout, self.x_shape[0], axis=0)

5. MatMul 节点

MatMul 节点执行的是矩阵的乘法运算，多用于 Affine 层中，即
$$
\mathbf{y} = \mathbf{x}\mathbf{W}
$$

后向计算时，关键就是弄清楚 $y_j$，$x_i$ 和 $w_{ij}$ 之间的关系

**实现要点**

因为`MatMul`是带有参数的，所以在学习过程中需要使用参数对应的梯度来更新参数本身，这就要求在`MatMul`需要自己保存梯度；这里使用了列表来保存梯度，而且使用**深拷贝**来完成赋值，即 `grad[0][...] = dW`，之所以这么做是和后续的训练过程密切相关；在训练时，会依次生成各层，对应的 `grads` 也只会生成这一次，如果这里使用深拷贝来修改 `grad` 的值，那么在整个模型层面，就可以使用一个大的 `model.grads` 列表将所有层的 `grads` 依次连接起来放入；如果这里不用深拷贝，每一层在更新自己的梯度时，自己 `grads` 中的元素就会直接指向 `dW` 所指向的内存地址，而 `model.grads` 中所存储的该层的 `grads` 还是保存的时初始化模型时的引用地址，这样就无法必须显式来做 `model.grads` 和 `layer.grads` 之间的同步；而深拷贝相当于固定了 `layer.grads` 中各个元素引用的内存地址，这样各层在更新自己的梯度时一定会放到 `model.grads` 对应的位置上，在训练时，只需要各层在后向传播时更新自己的 `layer.grads`，`model.grads` 就会自动得到更新，非常方便

In [ ]:
class MatMul():
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        """
        dW 存入梯度列表来更新参数
        dx 作为信号继续后向传播
        """
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        
        ## 注意这里使用了深拷贝
        ## 即将 dW 中的所有元素实际拷贝到 grads 中的指定位置
        self.grads[0][...] = dW
        
        return dx